In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# general
import pandas as pd
import numpy as np

# explore/ stat
import scipy.stats as stats

# visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

# support modules
import env
import acquire
import acquire_others
# import prepare
# import summarize

Acquire data from mySQL using the python module to connect and query. 

**Deliverable: create acquire.py**

- single dataframe

- content requirements:
> logerror
>
> all fields
>
> airconditioningtypeid
>
> all properties having transaction in 2017, 
>
> include only the last transaction for each properity = no duplicate property id's
>
> zestimate error 
>
> date of transaction
> 
> only include properties that include a latitude and longitude value

In [2]:
df = acquire.get_data_from_mysql()
df = acquire.basic_clean(df)

In [3]:
df.describe()

,logerror,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,"77,381.00","77,381.00",50.00,"77,381.00","77,381.00","49,672.00","76,772.00",614.00,"6,023.00","77,185.00",...,"77,118.00","17,560.00",172.00,"77,269.00","77,380.00","77,381.00","77,379.00","77,376.00","2,886.00","77,137.00"
mean,0.02,"1,495,138.90",679.72,2.30,3.05,6.53,2.32,66.00,"1,366.51","1,785.22",...,"1,968.64",1.43,1.00,"189,314.53","490,134.48","2,016.00","301,095.41","5,995.53",14.09,"60,496,733,646,264.24"
std,0.17,"860,907.05",689.70,1.00,1.14,1.72,0.98,0.00,671.31,954.05,...,23.79,0.54,0.00,"230,087.42","653,443.97",0.00,"492,596.03","7,622.84",2.19,"1,535,241,981,138.57"
min,-4.66,349.00,38.00,0.00,0.00,1.00,1.00,66.00,44.00,128.00,...,"1,824.00",1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00"
25%,-0.02,"752,070.00",273.00,2.00,2.00,6.00,2.00,66.00,955.00,"1,182.00",...,"1,953.00",1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.63",14.00,"60,373,109,005,002.00"
50%,0.01,"1,497,932.00",515.00,2.00,3.00,6.00,2.00,66.00,"1,257.00","1,543.00",...,"1,970.00",1.00,1.00,"136,499.00","358,975.50","2,016.00","203,372.00","4,450.69",15.00,"60,376,032,003,008.00"
75%,0.04,"2,240,535.00",796.50,3.00,4.00,8.00,3.00,66.00,"1,615.00","2,113.00",...,"1,987.00",2.00,1.00,"218,787.00","569,001.50","2,016.00","366,796.50","6,927.79",15.00,"60,590,423,251,008.00"
max,5.26,"2,982,274.00","3,560.00",18.00,16.00,12.00,18.00,66.00,"6,912.00","35,640.00",...,"2,016.00",6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 17052889 to 14447871
Data columns (total 64 columns):
county                          77381 non-null object
state                           77381 non-null object
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
id                              77381 non-null int64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null float64
finishedsquarefeet13            41 non-null float64
finishedsquarefeet15            3009 non-null float64
finishedsquarefe

In [5]:
pd.DataFrame(df.columns)

,0
0,county
1,state
2,logerror
3,transactiondate
4,id
5,basementsqft
6,bathroomcnt
7,bedroomcnt
8,buildingqualitytypeid
9,calculatedbathnbr


Summarize data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

**Deliverable: summary.py**

Objective

a function that takes in a dataframe 

returns a dataframe
- each row is an atttribute name, 
- the first column is the number of rows with missing values for that attribute, 
- the second column is percent of total rows that have missing values for that attribute. 

Run the function and document takeaways from this on how you want to handle missing values.

In [13]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing/rows
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}).sort_values(by = 'pct_rows_missing', ascending = False)
    pd.options.display.max_columns = 200
    return cols_missing

In [14]:
nulls_by_col(df)

,num_rows_missing,pct_rows_missing
buildingclassdesc,77366,1.00
finishedsquarefeet13,77340,1.00
storydesc,77331,1.00
storytypeid,77331,1.00
basementsqft,77331,1.00
yardbuildingsqft26,77311,1.00
fireplaceflag,77209,1.00
architecturalstyledesc,77175,1.00
typeconstructiondesc,77159,1.00
finishedsquarefeet6,76995,1.00


In [ ]:
# H
def nulls_by_col(df):
    number_missing = df.isnull().sum()
    pct_missing = (df.isnull().sum())/df.shape[0]
    rows_missing_df = pd.DataFrame({'num_rows_missing': number_missing, 'pct_rows_missing': pct_missing})
    return nulls_by_column_df

In [24]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing,\
                                 'pct_cols_missing': pct_cols_missing})\
                                .reset_index()\
                                .groupby(['num_cols_missing','pct_cols_missing'])\
                                .count()\
                                .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [3]:
def df_value_counts(df):
    counts = pd.Series([])
    for i, col in enumerate(df.columns.values):
        if df[col].dtype == 'object':
            col_count = df[col].value_counts()
        else:
            col_count = df[col].value_counts(bins=10, sort=False)
        counts = counts.append(col_count)
    return counts


def sam_df_value_counts(df):
    for col in df.columns:
        print(f'{col}:')
        if df[col].dtype == 'object':
            col_count = df[col].value_counts()
        else:
            if df[col].nunique() >= 35:
                col_count = df[col].value_counts(bins=10, sort=False)
            else:
                col_count = df[col].value_counts()
        print(col_count)

- Keep only single unit properties. (e.g. no duplexes, no land/lot, ...)
> not only filter unitcnt
>
> consider logics like: bed#, sqft, unit type...etc.
>
> identify those with unitcnt not defined

Single Unit Properties (JQ)

Single Family Residential = 52320

Residential General = 37

Rural Residence = 0

Mobile Home = 74

Manufactured, Modular, Prefabricated Homes = 58

Inferred Single Family Residential = 0

Bungalow = 0

In [ ]:

df.propertylandusedesc == ['Single Family Residential', 'Residentail General', 'Mobile Home', 'Manufactured, Modular, Prefabricated ']

In [ ]:
(df.propertylandusedesc == 'Single Family Residential').sum()